In [ ]:
#%pip install optuna-integration[pytorch_lightning]
#pip install --upgrade jupyter ipywidgets

# Importaciones generales
import pandas as pd
import numpy as np
import os
import warnings

# PyTorch y PyTorch Lightning
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, TQDMProgressBar

# Scikit-learn para preprocesamiento
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Optuna para optimización de hiperparámetros
import optuna
from optuna.integration import PyTorchLightningPruningCallback

# Visualización
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Configuraciones adicionales
warnings.filterwarnings('ignore')
pio.templates.default = "plotly_white"

# Configuración del dispositivo (GPU o CPU)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo seleccionado: {DEVICE}")

# Configuración para reproducibilidad
pl.seed_everything(42)

In [ ]:
# Cargar los datos
df_train = pd.read_parquet("../../Datos/df_train.parquet")
df_test = pd.read_parquet("../../Datos/df_test.parquet")

# --- Creación de la columna de fecha ---
# Para crear un índice de tiempo correcto, combinamos año y semana.
# Usamos el día 1 de la semana (%w=1) para consistencia.
df_train['date'] = pd.to_datetime(df_train['anio'].astype(str) + '-' + df_train['semana'].astype(str) + '-1', format='%Y-%W-%w')
df_train = df_train.sort_values(by=['id_bar', 'date']).reset_index(drop=True)

print("Datos de entrenamiento cargados y ordenados:")
print(df_train.head())

In [ ]:
# Agregación global
df_agg = df_train.groupby('date')['dengue'].sum().reset_index()

# Gráfico de la serie de tiempo agregada
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_agg['date'], y=df_agg['dengue'], mode='lines', name='Casos de Dengue Totales'))
fig.update_layout(
    title_text='Casos de Dengue Agregados a lo Largo del Tiempo',
    xaxis_title='Fecha',
    yaxis_title='Número de Casos de Dengue',
    template='plotly_white'
)
fig.show()

In [ ]:
# Muestra de 5 barrios
sample_barrios = df_train['id_bar'].unique()[:5]
fig = make_subplots(rows=len(sample_barrios), cols=1, shared_xaxes=True, subplot_titles=[f"Barrio {b}" for b in sample_barrios])

for i, bar_id in enumerate(sample_barrios):
    df_barrio = df_train[df_train['id_bar'] == bar_id]
    fig.add_trace(go.Scatter(x=df_barrio['date'], y=df_barrio['dengue'], mode='lines', name=f'Barrio {bar_id}'), row=i+1, col=1)

fig.update_layout(height=800, title_text="Casos de Dengue para una Muestra de Barrios")
fig.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt

# Usaremos la serie agregada para el análisis de autocorrelación
dengue_agg_series = df_agg.set_index('date')['dengue']

# Gráficos ACF y PACF
fig, axes = plt.subplots(1, 2, figsize=(16, 5))
plot_acf(dengue_agg_series, lags=52, ax=axes[0])
plot_pacf(dengue_agg_series, lags=52, ax=axes[1])
axes[0].set_title('Autocorrelation Function (ACF)')
axes[1].set_title('Partial Autocorrelation Function (PACF)')
plt.show()

In [ ]:
# --- Codificación de ESTRATO ---
df_train['ESTRATO'] = df_train['ESTRATO'].astype('category')
df_processed = pd.get_dummies(df_train, columns=['ESTRATO'], prefix='estrato')

# --- Selección de Variables ---
target_col = 'dengue'
# Excluimos variables de ID y tiempo que no son features directas
features_to_exclude = ['id', 'id_bar', 'anio', 'semana', 'date', target_col]
feature_cols = [col for col in df_processed.columns if col not in features_to_exclude]

# Separar features numéricas para escalado
numerical_features = df_train.select_dtypes(include=np.number).columns.tolist()
numerical_features.remove('id_bar')
numerical_features.remove('anio')
# 'semana' es cíclica, pero la trataremos como numérica por simplicidad aquí
# 'dengue' (target) se escalará por separado

# Las features numéricas reales para el escalador
features_to_scale = [f for f in feature_cols if f in numerical_features]

print(f"Número total de features: {len(feature_cols)}")
print(f"Features a escalar: {features_to_scale}")

In [ ]:
# --- División Temporal ---
train_df = df_processed[df_processed['anio'] < 2021]
val_df = df_processed[df_processed['anio'] == 2021]

print(f"Tamaño del set de entrenamiento: {len(train_df)}")
print(f"Tamaño del set de validación: {len(val_df)}")

# --- Escalado ---
# Escalador para las features
feature_scaler = StandardScaler()
train_df.loc[:, features_to_scale] = feature_scaler.fit_transform(train_df[features_to_scale])
val_df.loc[:, features_to_scale] = feature_scaler.transform(val_df[features_to_scale])
# Escalamos todo el dataframe para el entrenamiento final
df_processed.loc[:, features_to_scale] = feature_scaler.transform(df_processed[features_to_scale])


# Escalador para el target (muy importante para la predicción)
target_scaler = StandardScaler()
train_df.loc[:, [target_col]] = target_scaler.fit_transform(train_df[[target_col]])
val_df.loc[:, [target_col]] = target_scaler.transform(val_df[[target_col]])
df_processed.loc[:, [target_col]] = target_scaler.transform(df_processed[[target_col]])

print("\nEscalado completado.")

In [ ]:
class TimeSeriesDataset(Dataset):
    """
    Dataset personalizado para series de tiempo multivariadas.
    Genera secuencias de datos y sus correspondientes etiquetas.
    """
    def __init__(self, data, group_ids, feature_cols, target_col, sequence_length):
        self.data = data
        self.group_ids = group_ids
        self.feature_cols = feature_cols
        self.target_col = target_col
        self.sequence_length = sequence_length
        
        self.sequences = self._create_sequences()

    def _create_sequences(self):
        """
        Crea secuencias para cada grupo (id_bar).
        """
        seqs = []
        # Agrupamos por barrio para no crear secuencias que mezclen datos de distintos lugares
        for bar_id in self.group_ids:
            group_data = self.data[self.data['id_bar'] == bar_id]
            features = group_data[self.feature_cols].values
            target = group_data[self.target_col].values
            
            num_samples = len(group_data)
            if num_samples > self.sequence_length:
                for i in range(num_samples - self.sequence_length):
                    x = features[i:i + self.sequence_length]
                    y = target[i + self.sequence_length]
                    seqs.append((x, y))
        return seqs

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        x, y = self.sequences[idx]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# Ejemplo de uso (los DataLoaders se crearán dentro de la función de Optuna)
# ya que dependen del 'sequence_length' y 'batch_size'
print("Clase TimeSeriesDataset definida.")

In [ ]:
class LSTMForecaster(nn.Module):
    """
    Arquitectura del modelo LSTM.
    """
    def __init__(self, input_size, hidden_size, n_layers, dropout_prob):
        super(LSTMForecaster, self).__init__()
        
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=n_layers,
            batch_first=True,  # Importante: (batch, seq, feature)
            dropout=dropout_prob if n_layers > 1 else 0
        )
        
        # LayerNorm aplicado a la salida de la LSTM
        self.layer_norm = nn.LayerNorm(hidden_size)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # x shape: (batch_size, sequence_length, input_size)
        lstm_out, _ = self.lstm(x)
        
        # Tomamos la salida del último paso de tiempo
        last_time_step_out = lstm_out[:, -1, :]
        
        # Aplicamos normalización y dropout
        out = self.layer_norm(last_time_step_out)
        out = self.dropout(out)
        
        # Capa final para la predicción
        out = self.fc(out)
        
        return out.squeeze(-1) # Devolvemos un tensor de (batch_size)

print("Clase LSTMForecaster definida.")

In [ ]:
# Definición de la función objetivo para Optuna (VERSIÓN CORREGIDA)
def objective(trial: optuna.trial.Trial):
    # --- 1. Definir espacio de búsqueda de hiperparámetros ---
    params = {
        'sequence_length': trial.suggest_int('sequence_length', 8, 20),
        'hidden_size': trial.suggest_int('hidden_size', 32, 128, log=True),
        'n_layers': trial.suggest_int('n_layers', 1, 3),
        'dropout_prob': trial.suggest_float('dropout_prob', 0.1, 0.5),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True),
        'batch_size': trial.suggest_categorical('batch_size', [32, 64, 128]),
    }

    # --- 2. Crear Datasets y DataLoaders ---
    # Este paso es idéntico, pero lo repetimos por claridad
    try:
        train_dataset = TimeSeriesDataset(
            data=train_df, group_ids=train_df['id_bar'].unique(),
            feature_cols=feature_cols, target_col=target_col,
            sequence_length=params['sequence_length']
        )
        val_dataset = TimeSeriesDataset(
            data=val_df, group_ids=val_df['id_bar'].unique(),
            feature_cols=feature_cols, target_col=target_col,
            sequence_length=params['sequence_length']
        )
        
        # Si el dataset está vacío, la prueba no es válida
        if len(train_dataset) == 0 or len(val_dataset) == 0:
            return float('inf')

        train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True, num_workers=2, drop_last=True)
        val_loader = DataLoader(val_dataset, batch_size=params['batch_size'], shuffle=False, num_workers=2)
    except Exception as e:
        # Si algo falla en la creación (p.ej. secuencias inválidas), la prueba no es válida
        # print(f"Skipping trial due to DataLoader error: {e}") # Descomentar para depurar
        return float('inf')


    # --- 3. Instanciar modelo, optimizador y función de pérdida ---
    input_size = len(feature_cols)
    model = LSTMForecaster(
        input_size=input_size,
        hidden_size=params['hidden_size'],
        n_layers=params['n_layers'],
        dropout_prob=params['dropout_prob']
    ).to(DEVICE)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=params['learning_rate'])
    # Usamos MSE para optimizar (es más sensible a grandes errores), pero MAE para evaluar
    loss_fn_train = nn.MSELoss()
    loss_fn_val = nn.L1Loss() # MAE para validación

    # --- 4. Bucle de Entrenamiento y Validación Manual ---
    n_epochs = 30
    patience = 5
    best_val_mae = float('inf')
    patience_counter = 0

    for epoch in range(n_epochs):
        # Bucle de Entrenamiento
        model.train()
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(DEVICE), y_batch.to(DEVICE)
            
            optimizer.zero_grad()
            preds = model(x_batch)
            loss = loss_fn_train(preds, y_batch)
            loss.backward()
            optimizer.step()

        # Bucle de Validación
        model.eval()
        total_val_mae = 0
        with torch.no_grad():
            for x_batch, y_batch in val_loader:
                x_batch, y_batch = x_batch.to(DEVICE), y_batch.to(DEVICE)
                preds = model(x_batch)
                total_val_mae += loss_fn_val(preds, y_batch).item() * x_batch.size(0)
        
        # Calcular MAE promedio de validación
        avg_val_mae = total_val_mae / len(val_dataset)

        # --- Integración con Optuna (Pruning) y Early Stopping ---
        # 1. Reportar la métrica a Optuna
        trial.report(avg_val_mae, epoch)
        
        # 2. Verificar si la prueba debe ser podada
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        # 3. Lógica de Early Stopping
        if avg_val_mae < best_val_mae:
            best_val_mae = avg_val_mae
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
            break # Detener el entrenamiento si no hay mejora

    return best_val_mae

# --- Ejecutar el estudio de Optuna (sin cambios) ---
print("Iniciando búsqueda de hiperparámetros con Optuna (versión corregida)...")
study = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner())
# Aumentar n_trials para una búsqueda más exhaustiva (ej. 50-100). Usamos 20 por tiempo.
study.optimize(objective, n_trials=20, timeout=1800) # Timeout de 30 minutos

print("\nBúsqueda de hiperparámetros finalizada.")
print(f"Mejor MAE de validación: {study.best_value:.4f}")
print("Mejores hiperparámetros:")
print(study.best_params)

In [ ]:
# --- Obtener mejores parámetros ---
best_params = study.best_params

# --- Crear DataLoader con todos los datos de entrenamiento ---
final_sequence_length = best_params['sequence_length']
final_batch_size = best_params['batch_size']

full_train_dataset = TimeSeriesDataset(
    data=df_processed, # Usamos el dataframe procesado completo
    group_ids=df_processed['id_bar'].unique(),
    feature_cols=feature_cols,
    target_col=target_col,
    sequence_length=final_sequence_length
)

full_train_loader = DataLoader(full_train_dataset, batch_size=final_batch_size, shuffle=True, num_workers=2)

# --- Instanciar y entrenar el modelo final ---
final_model = LSTMForecaster(
    input_size=len(feature_cols),
    hidden_size=best_params['hidden_size'],
    n_layers=best_params['n_layers'],
    dropout_prob=best_params['dropout_prob']
).to(DEVICE)

optimizer = torch.optim.AdamW(final_model.parameters(), lr=best_params['learning_rate'])
loss_fn = nn.MSELoss()

print("\nEntrenando el modelo final sobre todos los datos...")
final_model.train()
num_epochs_final = 25 # Número de épocas para el entrenamiento final

for epoch in range(num_epochs_final):
    total_loss = 0
    for batch in full_train_loader:
        x, y = batch
        x, y = x.to(DEVICE), y.to(DEVICE)
        
        optimizer.zero_grad()
        preds = final_model(x)
        loss = loss_fn(preds, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(full_train_loader)
    print(f"Época {epoch+1}/{num_epochs_final} - Loss: {avg_loss:.6f}")

print("Entrenamiento final completado.")

In [ ]:
final_model.eval()
predictions = []
num_pred_weeks = 52 # Semanas a predecir para 2022

all_barrios = df_processed['id_bar'].unique()

with torch.no_grad():
    for bar_id in all_barrios:
        # 1. Obtener la última secuencia conocida para el barrio
        bar_data = df_processed[df_processed['id_bar'] == bar_id].tail(final_sequence_length)
        
        current_sequence_features = torch.tensor(bar_data[feature_cols].values, dtype=torch.float32).to(DEVICE)
        
        # Guardar las características exógenas de la última semana para llevarlas adelante
        last_week_exog_features = current_sequence_features[-1, :].clone()
        
        # 2. Bucle de predicción autorregresiva
        for week_num in range(1, num_pred_weeks + 1):
            # Preparar la entrada para el modelo
            input_tensor = current_sequence_features.unsqueeze(0) # (1, seq_len, num_features)
            
            # Predecir el siguiente valor (escalado)
            scaled_pred = final_model(input_tensor).item() # Sale como un escalar
            
            # Invertir el escalado para obtener el valor real de 'dengue'
            # Usamos reshape(-1, 1) ya que el scaler espera un array 2D
            unscaled_pred_array = target_scaler.inverse_transform(np.array([[scaled_pred]]))
            unscaled_pred = max(0, unscaled_pred_array[0, 0]) # Asegurar que los casos no sean negativos
            
            # Guardar la predicción
            submission_id = f"{bar_id}_2022_{week_num}"
            predictions.append({'id': submission_id, 'dengue': unscaled_pred, 'id_bar': bar_id, 'anio': 2022, 'semana': week_num})
            
            # 3. Construir la nueva característica para la semana predicha
            new_feature_row = last_week_exog_features.clone()
            
            # El target 'dengue' no es una feature, así que no necesitamos actualizarlo en el vector de features
            # Si el dengue de la semana anterior fuera una feature (lag), aquí se actualizaría.
            # En nuestro caso, el vector de features exógenas se mantiene constante.
            
            # 4. Actualizar la secuencia: quitar el primer paso, añadir el nuevo
            new_feature_row_expanded = new_feature_row.unsqueeze(0) # Shape: (1, num_features)
            current_sequence_features = torch.cat((current_sequence_features[1:, :], new_feature_row_expanded), dim=0)

print("Predicciones para 2022 generadas.")

# --- Crear y guardar el archivo de sumisión ---
submission_df = pd.DataFrame(predictions)
submission_df.to_csv("submission.csv", index=False)
print("\nArchivo submission.csv creado exitosamente.")
print(submission_df.head())

In [ ]:
# Preparar dataframe para visualización
pred_df = submission_df.copy()
pred_df['date'] = pd.to_datetime('2022-' + pred_df['semana'].astype(str) + '-1', format='%Y-%W-%w')

# Graficar para la misma muestra de barrios
fig = make_subplots(
    rows=len(sample_barrios), cols=1,
    shared_xaxes=True,
    subplot_titles=[f"Barrio {b}" for b in sample_barrios]
)

for i, bar_id in enumerate(sample_barrios):
    # Datos históricos
    hist_data = df_train[df_train['id_bar'] == bar_id]
    fig.add_trace(go.Scatter(
        x=hist_data['date'], y=hist_data['dengue'],
        mode='lines', name='Histórico', legendgroup='hist',
        showlegend=(i==0), line=dict(color='blue')
    ), row=i+1, col=1)

    # Datos de predicción
    pred_data = pred_df[pred_df['id_bar'] == bar_id]
    fig.add_trace(go.Scatter(
        x=pred_data['date'], y=pred_data['dengue'],
        mode='lines', name='Predicción 2022', legendgroup='pred',
        showlegend=(i==0), line=dict(color='red', dash='dash')
    ), row=i+1, col=1)

fig.update_layout(
    height=1000,
    title_text="Comparación de Datos Históricos vs. Predicciones para 2022",
    xaxis_title='Fecha',
    yaxis_title='Casos de Dengue'
)
fig.show()